In [142]:
import os
import pandas as pd
import numpy as np
import datetime as dt


def RSD(df_rsd):
    # returns travel time segment delay calculations.
    df_rsd['RSD'] =  df_rsd['SD'] - df_rsd['2017_EDT']
    df_rsd['RSD'] = np.where(df_rsd['RSD'] >= 0, df_rsd['RSD'], 0)
    return df_rsd
        
def segment_delay(df_sd):
    # returns SD spreadsheet value.
    df_sd['SD'] = df_sd['travel_time_seconds'].round()
    return df_sd

def AADT_splits(df_in):
    # apply ODOT methodology AADT_spit calculations to dataframe and create new columns.
    # df_in['aadt_singl] = Dir_AADT_BUD, 
    # df_in['aadt_combi'] = DirAADT_TRK
    df_in['DirAADT_AUTO'] = df_in['aadt'] - df_in['aadt_singl'] + df_in['aadt_combi']
    df_in['pct_auto'] = df_in['DirAADT_AUTO']/df_in['aadt']
    df_in['pct_bus'] = df_in['aadt_singl']/df_in['aadt']
    df_in['pct_truck'] = df_in['aadt_combi']/df_in['aadt']
    return df_in

def main():
    df = pd.read_csv(os.path.join(os.path.dirname('__file__'), 'Feb2017_test/Feb2017_test.csv')) #fix in script implementation
    # Filter by timestamps
    df['measurement_tstamp'] = pd.to_datetime(df['measurement_tstamp'])
    df = df[df['measurement_tstamp'].dt.hour.isin([6, 7, 8, 9, 16, 17, 18, 19])]
    df_meta = pd.read_csv(os.path.join(os.path.dirname('__file__'), 'Feb2017_test/TMC_Identification.csv'), 
                          usecols=['tmc', 'miles', 'tmclinear', 'aadt', 'aadt_singl', 'aadt_combi' ])
    df_odot = pd.read_csv(os.path.join(os.path.dirname('__file__'), 'Feb2017_test/odot_edt.csv'))    
    df = pd.merge(df, df_meta, left_on=df['tmc_code'], right_on=['tmc'], how='inner')
    df = pd.merge(df, df_odot, left_on=df['tmc_code'], right_on=['TMC'], how='inner')
    
    df = AADT_splits(df)
    df = segment_delay(df)
    df = RSD(df)
                                            
    # Return a selected row for test purposes only
    print(df.loc[df['tmc_code'] == '114-04368'])

In [143]:
main()

         tmc_code  measurement_tstamp  speed  average_speed  reference_speed  \
268800  114-04368 2017-02-01 06:00:00  43.64           52.0             62.0   
268801  114-04368 2017-02-01 06:15:00  29.11           52.0             62.0   
268802  114-04368 2017-02-01 06:30:00  32.24           52.0             62.0   
268803  114-04368 2017-02-01 06:45:00  22.42           52.0             62.0   
268804  114-04368 2017-02-01 07:00:00  27.39           53.0             62.0   
268805  114-04368 2017-02-01 07:15:00  23.10           53.0             62.0   
268806  114-04368 2017-02-01 07:30:00  29.32           53.0             62.0   
268807  114-04368 2017-02-01 07:45:00  26.76           53.0             62.0   
268808  114-04368 2017-02-01 08:00:00  29.97           51.0             62.0   
268809  114-04368 2017-02-01 08:15:00  27.83           51.0             62.0   
268810  114-04368 2017-02-01 08:30:00  23.82           51.0             62.0   
268811  114-04368 2017-02-01 08:45:00  2